<a href="https://colab.research.google.com/github/arielmeragelman/analisis_volatilidad_precios/blob/main/scrapper_lxml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install lxml
!pip install selenium
!pip install PyMySQL


# SOLO CUANDO SE INSTALA CROMIUM POR PRIMERA VEZ EN EL ENTORNO DE COLAB
!apt-get update #
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [17]:
# conexion con la base de dato local en casa


import pymysql 
link=pymysql.connect(
   host="192.141.210.12",    
   user="laboratorio", 
   password="laboratorio",
   db="bd_1", 
   charset="utf8",
   port=3306
   )


def consulta_lineas(sql,link):
  
  cursor = link.cursor()
  cursor.execute(sql)
  print(cursor.rowcount, "Record inserted successfully into Laptop table")
  for row in cursor.fetchall():
    print(row)
  cursor.close()


def insertar_linea(fecha,nombre_super,nombre_producto,precio_o,descuento,precio_final,categoria,link):
  sql = """insert into `entradas_super` (ID, Fecha, N_Super,N_Producto, Precio_Origen, Descuento, Precio_Final,Categoria)
         values (%s, %s, %s, %s, %s, %s, %s,%s) 
    """
  cursor = link.cursor()
  cursor.execute(sql,("",fecha,nombre_super,nombre_producto,precio_o,descuento,precio_final,categoria))
  link.commit()
  cursor.close()




from lxml import html
import requests
  
# Request the page

def scrap(web,categoria,xpath_nombre_a,xpath_nombre_b,xpath_precio_a,xpath_precio_b):
  
  
  page = requests.get(web, verify=False)
  tree = html.fromstring(page.content)  
  productos=[]  

  for elemento in range(1,30):
    print(elemento)
    print(xpath_nombre_a+str(elemento)+xpath_nombre_b)
    producto_n = tree.xpath(xpath_nombre_a+str(elemento)+xpath_nombre_b)[0]
    producto_n =producto_n.strip()
    producto_p = tree.xpath(xpath_precio_a+str(elemento)+xpath_precio_b)[0]
    producto_p =producto_p.strip()

    productos.append((producto_n,producto_p.replace("$", ""),categoria))
  #print(productos)
  return productos




def scrap_selenium(web,empresa,categoria,xpath_nombre_a,xpath_nombre_b,xpath_precio_a,xpath_precio_b):
  import sys
  import time
  from selenium.webdriver.support.ui import WebDriverWait
  from selenium.webdriver.common.by import By
  from selenium.webdriver.support import expected_conditions as EC


  sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
  from selenium import webdriver
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument("--start-maximized")
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

  productos=[]  
  
  wd.get(web)
  time.sleep(10)

  wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

  
  #test=" "
  #a= WebDriverWait(wd, 40).until(EC.visibility_of_element_located((By.XPATH, test))).text
  
  
  for elemento in range(1,30):
    print("XPATH NOMBRE ARTICULO:")
    print( xpath_nombre_a+str(elemento)+xpath_nombre_b)   
    
    try:
      producto_n = WebDriverWait(wd, 20).until(EC.visibility_of_element_located((By.XPATH, xpath_nombre_a+str(elemento)+xpath_nombre_b))).text
      print(producto_n)
    
      producto_n =producto_n.strip()
    except:
      print("FALLO EL PRODUCTO")
      break

    
    if empresa=="LIBERTAD":
      try:
        producto_p = WebDriverWait(wd, 30).until(EC.visibility_of_element_located((By.XPATH, xpath_precio_a+str(elemento)+xpath_precio_b))).text
        producto_p =producto_p.strip()
        print(producto_p)
  

      except:
        producto_p = WebDriverWait(wd, 20).until(EC.visibility_of_element_located((By.XPATH, xpath_precio_a+str(elemento)+xpath_precio_b+"[2]"))).text
        producto_p =producto_p.strip()
        print(producto_p)
    else:
      try:
        producto_p = WebDriverWait(wd, 30).until(EC.visibility_of_element_located((By.XPATH, xpath_precio_a+str(elemento)+xpath_precio_b))).text
        producto_p =producto_p.strip()
        print(producto_p)
      except:
        print("ERROR EN PRECIO")

    productos.append((producto_n,producto_p.replace("$", ""),categoria))
  return productos
  



#DEFINO LA FECHA DE EJECUCION 

from datetime import datetime
hoy=datetime.today().strftime('%Y-%m-%d')



#  "LIBERTAD"  "SUPER MAMI"
def obtengo_inserto(lista,empresa):

  for categoria in lista:
    if empresa == "SUPER MAMI":
      web='https://www.dinoonline.com.ar/super/categoria?_dyncharset=utf-8&Dy=1&Nty=1&minAutoSuggestInputLength=3&autoSuggestServiceUrl=%2Fassembler%3FassemblerContentCollection%3D%2Fcontent%2FShared%2FAuto-Suggest+Panels%26format%3Djson&searchUrl=%2Fsuper&containerClass=search_rubricator&defaultImage=%2Fimages%2Fno_image_auto_suggest.png&rightNowEnabled=false&Ntt='+categoria+''
      xpath_nombre_a='/html/body/div[4]/div/div[2]/div[4]/div['
      xpath_nombre_b=']/div/div[4]/text()'
      xpath_precio_a='/html/body/div[4]/div/div[2]/div[4]/div['
      xpath_precio_b=']/div/div[3]/span/text()' 
      
    
    elif empresa == "LIBERTAD":
      web='https://www.hiperlibertad.com.ar/busca?ft='+categoria+''
      xpath_nombre_a='/html/body/main/div[2]/div/div[3]/div/div[1]/div/div['
      xpath_nombre_b=']/div[2]/a[2]/h2'
      xpath_precio_a='/html/body/main/div[2]/div/div[3]/div/div[1]/div/div['
      xpath_precio_b=']/div[2]/div[1]/p'
      
    elif empresa == "CARREFOUR":
      web='https://www.carrefour.com.ar/'+categoria+'?_q='+categoria+''
      
      print(web)
      xpath_nombre_a='/html/body/div[3]/div/div[1]/div/div[6]/div/div/section/div[2]/div/div[2]/section/div/div/div/div[6]/div/div/div/div[2]/div['
      xpath_nombre_b=']/section/a/article/div/div[1]/div/div[2]/div/div[5]/div'
      xpath_precio_a='/html/body/div[3]/div/div[1]/div/div[6]/div/div/section/div[2]/div/div[2]/section/div/div/div/div[6]/div/div/div/div[2]/div['
      xpath_precio_b=']/section/a/article/div/div[1]/div/div[2]/div/div[3]/div/div/div/div/div[1]/span/span/span'


    
    try:
      respuesta_scrap=scrap(web,categoria,xpath_nombre_a,xpath_nombre_b,xpath_precio_a,xpath_precio_b)
    except:
      print("tiro error scrap")
      respuesta_scrap=scrap_selenium(web,empresa,categoria,xpath_nombre_a,xpath_nombre_b,xpath_precio_a,xpath_precio_b)
    for producto in respuesta_scrap:
        print(producto)
        print(hoy,empresa,producto[0],producto[1],"",producto[1],categoria,link)
        # descomentar cuando arregle lo de libertad
        insertar_linea(hoy,empresa,producto[0],producto[1],"",producto[1],categoria,link)
  






In [18]:


#cat_list=["acondicionador"]
cat_list=["leche","acondicionador","shampoo","jabon","desodorante","queso","cremoso","aceite","harina","detergente","empanada","papas","tregar"]

obtengo_inserto(cat_list,"LIBERTAD")




/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  method,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:79: DeprecationWarning: use options instead of chrome_options


1
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[1]/div[2]/a[2]/h2
tiro error scrap
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[1]/div[2]/a[2]/h2
LECHE PARCIALMENTE DESCREMADA UAT MANFREY LARGA VIDA 1 L
$99,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[2]/div[2]/a[2]/h2
LECHE ENTERA UAT MANFREY LARGA VIDA 1 L
$99,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[3]/div[2]/a[2]/h2
DULCE DE LECHE LA SERENÍSIMA CLASICO 400G
$172,14
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[4]/div[2]/a[2]/h2
CREMA DE LECHE LARGA VIDA ARMONIA 200CC
$103,89
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[5]/div[2]/a[2]/h2
DULCE DE LECHE CLASICO MANFREY 400 GR
$177,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[6]/div[2]/a[2]/h2
DULCE DE LECHE SAN IGNACIO TRADICION X 410 GR
$195,00
XPATH NOMBRE ARTICULO:
/html/body/mai

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  method,


1
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[1]/div[2]/a[2]/h2
tiro error scrap


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:79: DeprecationWarning: use options instead of chrome_options


...

XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[1]/div[2]/a[2]/h2
PAPAS FRITAS LAYS CLÁSICAS 379 GR
$660,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[2]/div[2]/a[2]/h2
PAPAS FRITAS LAYS CLÁSICAS 145 GR
$323,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[3]/div[2]/a[2]/h2
PAPAS FRITAS LAYS CLÁSICAS 45 GR
$135,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[4]/div[2]/a[2]/h2
PURÉ DE PAPAS MORIXE X 125 GR
$97,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[5]/div[2]/a[2]/h2
PAPAS MC CAIN TRADICIONAL BASTON CONG 400 GRS
$209,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[6]/div[2]/a[2]/h2
PAPAS MC CAIN TRADICIONALES BASTON CONG 720 GRS
$238,47
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[7]/div[2]/a[2]/h2
PURE DE PAPAS INSTANTANEO MAGGI ORIGINAL X 200 GR
$296,99
XPATH NOMBRE

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  method,


1
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[1]/div[2]/a[2]/h2
tiro error scrap


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:79: DeprecationWarning: use options instead of chrome_options


XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[1]/div[2]/a[2]/h2
RICOTA ENTERA TREGAR 290G
$187,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[2]/div[2]/a[2]/h2
QUESO BLANCO TREGAR LIGHT 300GR
$243,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[3]/div[2]/a[2]/h2
QUESO BLANCO TREGAR CLASICO 300GR
$243,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[4]/div[2]/a[2]/h2
ARROZ CON LECHE TREGAR 180GR
$85,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[5]/div[2]/a[2]/h2
QUESO RALLADO TREGAR X 40 GRS
$119,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[6]/div[2]/a[2]/h2
YOG.DES.VAI.TREGAR F 125 GRS
$68,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[7]/div[2]/a[2]/h2
Y.ENT DDL.TREGAR F 125 GRS
$68,99
XPATH NOMBRE ARTICULO:
/html/body/main/div[2]/div/div[3]/div/div[1]/div/div[8]/div

In [19]:
sql = """select * from entradas_super where categoria="leche" """

consulta_lineas(sql,link)

52 Record inserted successfully into Laptop table
(4, datetime.date(2022, 4, 11), 'SUPER MAMI', 'ALIMENTO LA SERENISIMA A BASE DE ALMENDRAS BRIK X ', 410.8, 0.0, 410.8, 'leche')
(5, datetime.date(2022, 4, 11), 'SUPER MAMI', 'ALIMENTO LA SERENISIMA A BASE DE ARROZ BRIK X 1 LT', 384.8, 0.0, 384.8, 'leche')
(6, datetime.date(2022, 4, 11), 'SUPER MAMI', 'ALIMENTO LIQUIDO A BASE DE ALMENDRA SILK 946 ML NA', 270.0, 0.0, 270.0, 'leche')
(7, datetime.date(2022, 4, 11), 'SUPER MAMI', 'ALIMENTO LIQUIDO A BASE DE ALMENDRA SILK 946 ML NA', 270.0, 0.0, 270.0, 'leche')
(8, datetime.date(2022, 4, 11), 'SUPER MAMI', 'ALIMENTO LIQUIDO A BASE DE ALMENDRA SILK 946 ML VA', 270.0, 0.0, 270.0, 'leche')
(9, datetime.date(2022, 4, 11), 'SUPER MAMI', 'BEBIDA LACTEA A BASE LA SERENISIMA SENSE CAPPUCHIN', 130.0, 0.0, 130.0, 'leche')
(10, datetime.date(2022, 4, 11), 'SUPER MAMI', 'BEBIDA LACTEA A BASE LA SERENISIMA SENSE CARAMEL X', 317.2, 0.0, 317.2, 'leche')
(11, datetime.date(2022, 4, 11), 'SUPER MAMI', 'BEBID